# For Base Rate

In [1]:
# 노가다 10년안에바뀐횟수 <20회

# For Call Rate

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
f = open("call.csv", "w")
f.write("날짜,콜금리,\n")
for i in range(1,400): 
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page='+str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    list_call = soup.select('div[class="section_exchange2"] tbody > tr')
    for call in list_call:
        date=str(call.select('td')[0].text.strip())
        call_index=str(call.select('td')[1].text.strip())
        f.write(str(date)+","+str(call_index)+'\n')
f.close()

# For Bond Analyst Reports

In [ ]:
# option 1
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
!pip install openpyxl
import openpyxl
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["날짜","제목","본문"])
for i in range(1,101): # 마지막 페이지 입력하기 101
    URL = 'https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2011-01-01&writeToDate=2021-12-31&x=0&y=0&page='+str(i)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    list_minutes = soup.select('div[id="contentarea_left"] div[class="box_type_m"] tr')
    for minutes in list_minutes:
        try:
            title=minutes.select('a')[0].text
            sub_link = minutes.select('a')[1].attrs['href']
            date = datetime.strptime('20'+ minutes.select('.date')[0].text, "%Y.%m.%d")
            if str(sub_link).endswith('.pdf'):
                print(date.strftime('%Y.%m.%d'))
                print(title)
                print(sub_link)
                file_res = requests.get(sub_link)
                sheet.append([ds.strftime('%Y.%m.%d'),title,sub_link])
            else:
                pass
        except:
            pass
wb.save("bond_report.csv")

In [ ]:
# option 2 - x download pdf/ getting urls only
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import csv
f = open('bond_report.csv','w',newline='')
wr = csv.writer(f)
for i in range(1,102): # 마지막 페이지+1 입력
    URL = 'https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2011-01-01&writeToDate=2021-12-31&x=0&y=0&page='+str(i)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    list_minutes = soup.select('div[id="contentarea_left"] div[class="box_type_m"] tr')
    for minutes in list_minutes:
        try:
            title=minutes.select('a')[0].text
            sub_link = minutes.select('a')[1].attrs['href']
            date = datetime.strptime('20'+ minutes.select('.date')[0].text, "%Y.%m.%d")
            date_convert = date.strftime('%Y.%m.%d')
            print(date_convert)
            print(title)
            print(sub_link)
            wr.writerow([date_convert,title,sub_link])
        except:
            pass
f.close()

In [ ]:
#링크안 pdf text 긁어오기
import os
from tika import parser
import pandas as pd
import csv
bond_report = pd.read_csv("bond_report.csv", header=0)
bond_report.columns = ['date','title','url']
content = []
for url in bond_report['url']:
    try:
        content.append(parser.from_file(url)['content'].replace('\n', ''))
    except:
        content.append("NA")
        pass
bond_report['content'] = content
bond_report 
bond_report.to_csv(r'/Users/soojinkim/kdigital/mini_project_1/bond_report_df.csv')

# For Monetary Policy Board Minutes

## To Download PDF files:

In [ ]:
#금통위 의사록 다운로드 최종
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
import re
for i in range(1,21): # 마지막 페이지 입력하기
    URL = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex='+str(i)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    list_minutes = soup.select('main[id="content"] div[class="bdLine type2"] > ul > li')
    for minutes in list_minutes:
        title = minutes.select('a')[0].text.strip()
        pdf_check = minutes.select('a')[2].text.strip()
        pdf_check2 = minutes.select('a')[4].text.strip()
        sub_link = minutes.select('a')[2].attrs['href']
        sub_link2 = minutes.select('a')[4].attrs['href']
        if re.findall(r'(?<=차\)).*',title)[0][-2] != '.':  # (1111.11.11)형식이면
            pdf_date = datetime.strptime(re.findall(r'(?<=차\)).*',title)[0][1:-1],"%Y.%m.%d")
            pdf_date2 = pdf_date.strftime('%Y.%m.%d')
        else:
            pdf_date = datetime.strptime(re.findall(r'(?<=차\)).*',title)[0][1:-2],"%Y.%m.%d")
            pdf_date2 = pdf_date.strftime('%Y.%m.%d')
        if 'pdf' in pdf_check:
            link = 'https://www.bok.or.kr' + sub_link
        else:
            link = 'https://www.bok.or.kr' + sub_link2
        file_res = requests.get(link)
        with open(f"{pdf_date2}.pdf", 'wb') as f:
            f.write(file_res.content)

## To Extract Content from PDF Files:

In [ ]:
from tika import parser
from datetime import datetime, timedelta, date
f = open('mp_report.csv','w',newline='')
wr = csv.writer(f)
start_date = date(2011,1,1)
end_date = date(2020,12,31)
ds = start_date
while True:
    try:
        filename = f'./금융통화위원회 의사록/{ds.strftime("%Y.%m.%d")}.pdf'
        parsed = parser.from_file(filename, requestOptions={'timeout':120})
#        print(ds)
#        print(parsed['content'].strip())
        wr.writerow([ds,parsed['content'].strip()])
    except:
        pass
    ds += timedelta(1)
    if ds == end_date:
        break
f.close()
pd.read_csv('mp_report.csv',header=None)

# For Naver News Articles

In [ ]:
import openpyxl
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["날짜","기사명","본문"])
#은이님 수정
from bs4 import BeautifulSoup
import requests
import csv
import re
import datetime
start_date = datetime.datetime(2013,1,1)
end_date = datetime.datetime(2014,1,1)
total_days = int((end_date-start_date).days)
ds=start_date #시작일자
errorcount=0
for day in range(1,total_days+1):
    page = 1
    while True:
        headers = {'user-agent':'Mozilla/5.0','cookie':'news_office_checked=1001,1018,2227'}
        url = f"https://search.naver.com/search.naver?&where=news&query=%EA%B8%88%EB%A6%AC&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={ds.strftime('%Y.%m.%d')}&de={ds.strftime('%Y.%m.%d')}&docid=&nso=so:r,p:from{ds.strftime('%Y%m%d')}to{ds.strftime('%Y%m%d')},a:all&mynews=1&start={page}&refresh_start=0"
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text,"html.parser")        
        li_list = soup.select(".list_news > li")
        for li in li_list:
            try:
#                print(ds.strftime('%Y.%m.%d'))
                if li.select('div .news_area a[href]')[0]['href'] == 'http://www.einfomax.com':
                    title = li.select('div .news_area a[title]')[0].text
                    article_url = li.select('div .news_area a[title]')[0]['href']
                    article_response = requests.get(article_url, headers=headers)
                    article_soup = BeautifulSoup(article_response.text,"html.parser")
                    article = article_soup.find(id = 'article-view-content-div')
                    body = re.sub(r'[\n\t]','',article.text.strip())
                else:
                    title = li.select('div .news_area a[title]')[0].text
                    article_url = li.select('div .news_area a[href]')[1]['href']
                    article_response = requests.get(article_url, headers=headers)
                    article_soup = BeautifulSoup(article_response.text,"html.parser")
                    article = article_soup.find(id = 'articleBodyContents')
                    body = article.text.strip()
#                 print(title)
#                 print(body)
#                 print("") 
                sheet.append([ds.strftime('%Y.%m.%d'),title,body])
            except:
                errorcount+=1
        button=soup.select('.sc_page > a')[1]['aria-disabled']
        page+=10
        if button == 'true':
            ds+=datetime.timedelta(1)
            break
wb.save("2020.csv")